# 🚀 Hierarchical Subchat System - Kaggle GPU Testing

## 📋 Setup Checklist (Do Once):

### 1️⃣ **Add Kaggle Secrets** (Most Important!)
Go to: **https://www.kaggle.com/settings** → Add-ons → Secrets

Add these two secrets:
- **`GROQ_API_KEY`** = Your Groq API key (for query decomposition)
- **`GITHUB_TOKEN`** = Your GitHub personal access token (for pushing results)

### 2️⃣ **Enable Internet in This Notebook**
- Click "⚙️ Settings" (top right)
- Turn ON **"Internet"** toggle
- Click "Save"

### 3️⃣ **Make Sure This Notebook is PRIVATE**
- Never share secrets in public notebooks!

---

## ▶️ Run Order:
1. **Cells 2-7**: Load secrets and libraries
2. **Cell 8**: Set environment variables (LLM_BACKEND=vllm)
3. **Cell 9**: Load vLLM model on Kaggle GPUs (Qwen-3 14B AWQ) - **Takes 2-3 minutes**
4. **Cells 15-17**: Clone repo, configure git, run test log push
5. **Cell 21**: Integrate vLLM with backend (registers model globally)
6. **Cell 23**: Test vLLM integration
7. **Run your actual tests**: Execute test scripts in backend/dataset/
8. **Push results**: Use git_commit_and_push() function to sync to GitHub

---

## 🎯 What This Notebook Does:
- ✅ Loads vLLM model (Qwen-3 14B) on Kaggle's 2x GPUs with AWQ quantization
- ✅ Integrates vLLM with your backend via `VLLMClient` singleton
- ✅ Runs hierarchical subchat tests using GPU-accelerated inference
- ✅ Generates performance logs for buffer size analysis
- ✅ Automatically syncs results back to your GitHub repo

---

## 🔧 Technical Details:
- **vLLM Config**: Tensor parallelism across both GPUs, 91% memory utilization
- **Backend Mode**: `LLM_BACKEND=vllm` (set in cell 8)
- **Model**: Qwen-3 14B AWQ (5120 max tokens, prefix caching enabled)
- **No .env needed**: Secrets loaded from Kaggle environment
- **Git Workflow**: Clone → Test → Push results to `kaggle-run` branch

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/config.json
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/merges.txt
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/LICENSE
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/README.md
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/tokenizer.json
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/vocab.json
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/tokenizer_config.json
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/model.safetensors
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/.gitattributes
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/generation_config.json
/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1/model.safetensors.index.json
/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1/model-00003-of-00003.safetensors
/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1/config.json
/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1/merges.txt
/kag

# CHECKING OUT MY GPU WORKINGW

*** repo: https://github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM ***


In [2]:
! uv pip uninstall -q --system 'tensorflow'
! uv pip install -q --system 'vllm' 'triton==3.2.0' 'logits-processor-zoo' 'numpy<2'

In [3]:
import os
import re
import logging
from pathlib import Path
import pickle
import json
import joblib
import shutil
import glob
from tqdm.auto import tqdm
import warnings

import numpy as np
import pandas as pd



# For Qwen
import torch
import vllm
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor


INFO 12-18 15:10:59 [__init__.py:239] Automatically detected platform cuda.


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GITHUB_TOKEN")
secret_value_1 = user_secrets.get_secret("GROQ_API_KEY")
secret_value_2 = user_secrets.get_secret("HuggingFACEHUB_access_token")
secret_value_3 = user_secrets.get_secret("LANGCHAIN_API_KEY")

# ✅ IMPORTANT: Set them in os.environ so other code can access them
os.environ["GITHUB_TOKEN"] = secret_value_0
os.environ["GROQ_API_KEY"] = secret_value_1
os.environ["HuggingFACEHUB_access_token"] = secret_value_2
os.environ["LANGCHAIN_API_KEY"] = secret_value_3
os.environ["LLM_BACKEND"] = "vllm"

# ✅ NEW: Set vLLM model path for backend config
# This will be used by config.py when backend starts
model_path = "/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1"
os.environ["VLLM_MODEL_PATH"] = model_path

# Print the tokens (first 4 and last 4 characters for security)
print("="*60)
print("🔐 SECRETS LOADED AND SET IN ENVIRONMENT")
print("="*60)
print(f"✅ GITHUB_TOKEN: {secret_value_0[:4]}...{secret_value_0[-4:]}")
print(f"✅ GROQ_API_KEY: {secret_value_1[:4]}...{secret_value_1[-4:]}")
print(f"✅ HuggingFACEHUB_access_token: {secret_value_2[:4]}...{secret_value_2[-4:]}")
print(f"✅ LANGCHAIN_API_KEY: {secret_value_3[:4]}...{secret_value_3[-4:]}")
print(f"✅ LLM_BACKEND: vllm")
print(f"✅ VLLM_MODEL_PATH: {model_path}")
print("="*60)

🔐 SECRETS LOADED AND SET IN ENVIRONMENT
✅ GITHUB_TOKEN: gith...tWfg
✅ GROQ_API_KEY: gsk_...l6gr
✅ HuggingFACEHUB_access_token: hf_E...GaQC
✅ LANGCHAIN_API_KEY: lsv2...ea2f
✅ LLM_BACKEND: vllm
✅ VLLM_MODEL_PATH: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1


In [5]:
# vLLM V1 does not currently accept logits processor so we need to disable it
# https://docs.vllm.ai/en/latest/getting_started/v1_user_guide.html#deprecated-features
os.environ["VLLM_USE_V1"] = "0"

# Use 14B model (32B causes CUDA linker failures on Kaggle T4 GPUs)
#model_path = "/kaggle/input/qwen-3/transformers/14b-awq/1"
llm = vllm.LLM(
    model_path,
    quantization='awq',
    tensor_parallel_size=torch.cuda.device_count(),
    gpu_memory_utilization=0.91,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True,
    enable_prefix_caching=True
)
tokenizer = llm.get_tokenizer()

INFO 12-18 15:11:29 [config.py:717] This model supports multiple tasks: {'embed', 'classify', 'score', 'generate', 'reward'}. Defaulting to 'generate'.
WARNING 12-18 15:11:30 [config.py:830] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 12-18 15:11:30 [config.py:830] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-18 15:11:30 [config.py:1770] Defaulting to use mp for distributed inference
WARNING 12-18 15:11:30 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 12-18 15:11:30 [config.py:1770] Defaulting to use mp for distributed inference
WARNING 12-18 15:11:30 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 12-18 15:11:30 [llm_engine.py:240] Initializing a V0 LLM engine (

[W1218 15:11:49.581988777 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1218 15:11:49.582745975 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 12-18 15:11:49 [utils.py:1055] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=158) INFO 12-18 15:11:49 [utils.py:1055] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=158) INFO 12-18 15:11:49 [pynccl.py:69] vLLM is using nccl==2.21.5
INFO 12-18 15:11:49 [pynccl.py:69] vLLM is using nccl==2.21.5
INFO 12-18 15:11:49 [pynccl.py:69] vLLM is using nccl==2.21.5


[W1218 15:11:49.861028203 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1218 15:11:49.861695448 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 12-18 15:11:49 [custom_all_reduce_utils.py:206] generating GPU P2P access cache in /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 12-18 15:12:14 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=158) INFO 12-18 15:12:14 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 12-18 15:12:14 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=158) INFO 12-18 15:12:14 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 12-18 15:12:15 [shm_broadcast.py:266] vLLM message queue communication handle: Handle(local_reader_ranks=[1], buffer_handle=(1, 4194304, 6, 'psm_4c80dd20'), local_subscribe_addr='ipc:///tmp/2d6b6314-3cc8-47a6-b116-baee3e8f0637', remote_subscri

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


(VllmWorkerProcess pid=158) INFO 12-18 15:13:33 [loader.py:458] Loading weights took 77.73 seconds
INFO 12-18 15:13:33 [loader.py:458] Loading weights took 77.84 seconds
(VllmWorkerProcess pid=158) INFO 12-18 15:13:33 [model_runner.py:1140] Model loading took 4.6720 GiB and 78.056700 seconds
INFO 12-18 15:13:33 [model_runner.py:1140] Model loading took 4.6720 GiB and 78.167646 seconds
(VllmWorkerProcess pid=158) INFO 12-18 15:13:33 [model_runner.py:1140] Model loading took 4.6720 GiB and 78.056700 seconds
INFO 12-18 15:13:33 [model_runner.py:1140] Model loading took 4.6720 GiB and 78.167646 seconds
(VllmWorkerProcess pid=158) INFO 12-18 15:13:44 [worker.py:287] Memory profiling takes 10.84 seconds
(VllmWorkerProcess pid=158) INFO 12-18 15:13:44 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.91) = 13.41GiB
(VllmWorkerProcess pid=158) INFO 12-18 15:13:44 [worker.py:287] model weights take 4.67GiB; non_torch_memory takes 0.12GiB; 

In [6]:
from vllm import SamplingParams

def stream_generate(llm, prompt):
    sampling_params = SamplingParams(
        temperature=0.2,
        top_p=0.9,
        max_tokens=512,
    )

    for output in llm.generate(
        [prompt],
        sampling_params,
        
    ):
        yield output.outputs[0].text


# ── Usage ────────────────────────────────────────────────
prompt = """You are a helpful assistant.
User: Explain tensor parallelism in simple terms.
Assistant:"""

for token in stream_generate(llm, prompt):
    print(token, end="", flush=True)


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 Tensor parallelism is a technique used in deep learning and machine learning to speed up the processing of large datasets and complex models. Imagine you have a big puzzle that you need to solve, but it's too big for one person to handle. So, you decide to split the puzzle into smaller pieces and distribute them among several friends. Each friend works on their piece independently, and then you put all the pieces together to get the final solution.

In the context of machine learning, a "tensor" is like a multi-dimensional array or matrix that holds data. Tensor parallelism works similarly by splitting these tensors into smaller parts and distributing them across multiple processors or machines. Each processor works on its part of the tensor, performing calculations in parallel, and then combines the results to get the final output.

This approach helps in reducing the time needed to train large models and process big datasets by leveraging the power of multiple processors working sim

In [7]:
prompt = """what is quantum computing?"""

for token in stream_generate(llm, prompt):
    print(token, end="", flush=True)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. In contrast to classical computers, which use bits to represent information, quantum computers use quantum bits, or qubits. This allows quantum computers to perform certain types of calculations much faster than classical computers. Quantum computing has the potential to revolutionize fields such as cryptography, optimization, and simulation. However, it is still a relatively new and rapidly evolving field, and there are many technical challenges that need to be overcome before quantum computers can be widely used. 

In more detail, a quantum computer can process a vast amount of data simultaneously due to the principles of superposition and entanglement. Superposition allows a qubit to exist in multiple states at the same time, while entanglement allows qubits to be connected in such a way that the state of one qubit can instantly aff

In [8]:
print('h')

h


# test github

In [9]:
# Load secrets from Kaggle's secure environment
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

print("="*60)
print("🔐 LOADING SECRETS FROM KAGGLE")
print("="*60)

# Try to load GROQ_API_KEY
try:
    GROQ_API_KEY = user_secrets.get_secret("GROQ_API_KEY")
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY
    print("✅ GROQ_API_KEY loaded successfully")
    print(f"   Key length: {len(GROQ_API_KEY)} characters")
except Exception as e:
    print(f"⚠️  GROQ_API_KEY not found: {e}")
    print("   Add it in Kaggle Settings → Secrets")
    GROQ_API_KEY = None

# Try to load GITHUB_TOKEN
try:
    GITHUB_TOKEN = user_secrets.get_secret("GITHUB_TOKEN")
    os.environ["GITHUB_TOKEN"] = GITHUB_TOKEN
    print("✅ GITHUB_TOKEN loaded successfully")
    print(f"   Token length: {len(GITHUB_TOKEN)} characters")
except Exception as e:
    print(f"⚠️  GITHUB_TOKEN not found: {e}")
    print("   Add it in Kaggle Settings → Secrets")
    GITHUB_TOKEN = None

# Set LLM backend to use vLLM (local model on Kaggle GPU)
os.environ["LLM_BACKEND"] = "vllm"  # We'll use the vLLM model loaded above
print("\n✅ LLM_BACKEND set to 'vllm' (using Kaggle GPU)")

print("="*60)

🔐 LOADING SECRETS FROM KAGGLE
✅ GROQ_API_KEY loaded successfully
   Key length: 56 characters
✅ GITHUB_TOKEN loaded successfully
   Token length: 93 characters

✅ LLM_BACKEND set to 'vllm' (using Kaggle GPU)


In [10]:
# Check GPU availability and configuration
import torch

print("="*60)
print("🔍 ENVIRONMENT CHECK")
print("="*60)
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
print(f"✅ CUDA version: {torch.version.cuda}")
print(f"✅ Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"\n🎮 GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"   Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")

print(f"\n✅ Current working directory: {os.getcwd()}")
print("="*60)

🔍 ENVIRONMENT CHECK
✅ PyTorch version: 2.6.0+cu124
✅ CUDA available: True
✅ CUDA version: 12.4
✅ Number of GPUs: 2

🎮 GPU 0: Tesla T4
   Memory: 14.74 GB

🎮 GPU 1: Tesla T4
   Memory: 14.74 GB

✅ Current working directory: /kaggle/working


In [11]:
# Clone the kaggle-run branch from GitHub (PUBLIC READ - no auth needed)
import subprocess

REPO_URL = "https://github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM.git"
REPO_DIR = "Subchat-Trees"
BRANCH = "kaggle-run"

print("="*60)
print("📥 CLONING REPOSITORY")
print("="*60)

# Remove existing directory if present
if os.path.exists(REPO_DIR):
    print(f"⚠️  Removing existing {REPO_DIR} directory...")
    shutil.rmtree(REPO_DIR)

# Clone the specific branch (no authentication needed for public repos)
# Skip LFS files to avoid bandwidth quota issues
print(f"🔄 Cloning {BRANCH} branch (skipping LFS files)...")
print("   No authentication required for cloning (public repo)")

# Set environment variable to skip LFS files
clone_env = os.environ.copy()
clone_env["GIT_LFS_SKIP_SMUDGE"] = "1"

result = subprocess.run(
    ["git", "clone", "-b", BRANCH, "--single-branch", REPO_URL, REPO_DIR],
    capture_output=True,
    text=True,
    env=clone_env
)

if result.returncode == 0:
    print(f"✅ Successfully cloned {BRANCH} branch!")
    print(f"📂 Repository location: {os.path.abspath(REPO_DIR)}")
    
    # Pull Git LFS files for scenarios only (saves bandwidth)
    print("\n📥 Pulling Git LFS scenario files...")
    os.chdir(REPO_DIR)
    lfs_result = subprocess.run(
        ["git", "lfs", "pull", "--include=backend/dataset/scenarios/*.json"],
        capture_output=True,
        text=True
    )
    
    if lfs_result.returncode == 0:
        print("✅ Successfully pulled scenario files from Git LFS")
    else:
        print(f"⚠️  Warning: Git LFS pull returned code {lfs_result.returncode}")
        if lfs_result.stderr:
            print(f"   {lfs_result.stderr}")
    
    os.chdir("..")  # Return to parent directory
    
    # List key directories to verify
    print("\n📁 Key directories found:")
    key_dirs = ["backend", "backend/src", "backend/dataset"]
    for dir_path in key_dirs:
        full_path = os.path.join(REPO_DIR, dir_path)
        if os.path.exists(full_path):
            print(f"   ✅ {dir_path}")
        else:
            print(f"   ❌ {dir_path} (not found)")
else:
    print(f"❌ Clone failed: {result.stderr}")
    
print("="*60)

📥 CLONING REPOSITORY
🔄 Cloning kaggle-run branch (skipping LFS files)...
   No authentication required for cloning (public repo)
✅ Successfully cloned kaggle-run branch!
📂 Repository location: /kaggle/working/Subchat-Trees

📥 Pulling Git LFS scenario files...
✅ Successfully pulled scenario files from Git LFS

📁 Key directories found:
   ✅ backend
   ✅ backend/src
   ✅ backend/dataset


In [12]:
# Configure git identity
os.chdir(REPO_DIR)

print("="*60)
print("⚙️  CONFIGURING GIT")
print("="*60)

!git config user.name "moonmehedi"
!git config user.email "the.mehedi.hasan.moon@gmail.com"

print("✅ Git identity configured!")
print(f"   User: moonmehedi")
print(f"   Email: the.mehedi.hasan.moon@gmail.com")

# Verify current branch
branch_result = subprocess.run(["git", "branch", "--show-current"], capture_output=True, text=True)
print(f"\n✅ Current branch: {branch_result.stdout.strip()}")
print("="*60)

os.chdir("..")  # Return to parent directory

⚙️  CONFIGURING GIT


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✅ Git identity configured!
   User: moonmehedi
   Email: the.mehedi.hasan.moon@gmail.com

✅ Current branch: kaggle-run


In [13]:
def create_test_log(log_dir="kaggle_logs", log_file="connection_test.log"):
    """Create a detailed test log with GPU and environment info"""
    from datetime import datetime
    
    os.makedirs(log_dir, exist_ok=True)
    log_path = os.path.join(log_dir, log_file)
    current_time = datetime.now()
    
    with open(log_path, "w") as f:
        f.write("="*60 + "\n")
        f.write("🔬 KAGGLE GPU TEST RUN - CONNECTION VERIFICATION\n")
        f.write("="*60 + "\n\n")
        
        f.write(f"📅 Test Date: {current_time.strftime('%Y-%m-%d')}\n")
        f.write(f"⏰ Test Time: {current_time.strftime('%H:%M:%S UTC')}\n")
        f.write(f"📍 Timestamp: {pd.Timestamp.now()}\n\n")
        
        f.write("="*60 + "\n")
        f.write("🎮 GPU CONFIGURATION\n")
        f.write("="*60 + "\n")
        f.write(f"GPU Count: {torch.cuda.device_count()}\n")
        
        if torch.cuda.is_available():
            for i in range(torch.cuda.device_count()):
                f.write(f"\nGPU {i}:\n")
                f.write(f"  - Name: {torch.cuda.get_device_name(i)}\n")
                f.write(f"  - Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB\n")
        else:
            f.write("⚠️  No GPU detected\n")
        
        f.write("\n" + "="*60 + "\n")
        f.write("📊 ENVIRONMENT INFO\n")
        f.write("="*60 + "\n")
        f.write(f"PyTorch Version: {torch.__version__}\n")
        f.write(f"CUDA Available: {torch.cuda.is_available()}\n")
        f.write(f"Working Directory: {os.getcwd()}\n")
        
        f.write("\n" + "="*60 + "\n")
        f.write("✅ TEST STATUS: SUCCESS\n")
        f.write("="*60 + "\n")
        f.write(f"\nThis log was generated from Kaggle notebook\n")
        f.write(f"Push attempt at: {current_time.isoformat()}\n")
    
    return log_path, current_time


def git_commit_and_push(file_path, commit_message, branch="kaggle-run"):
    """Commit a file and push to GitHub"""
    import subprocess
    
    # Add file
    add_result = subprocess.run(["git", "add", file_path], capture_output=True, text=True)
    if add_result.returncode != 0:
        return False, f"Git add failed: {add_result.stderr}"
    
    # Commit
    commit_result = subprocess.run(["git", "commit", "-m", commit_message], capture_output=True, text=True)
    if commit_result.returncode != 0:
        return False, f"Git commit failed: {commit_result.stderr}"
    
    # Push with token
    if "GITHUB_TOKEN" not in os.environ:
        return False, "GITHUB_TOKEN not found in environment"
    
    repo_url_with_token = f"https://{os.environ['GITHUB_TOKEN']}@github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM.git"
    
    # Set remote URL
    subprocess.run(["git", "remote", "set-url", "origin", repo_url_with_token], capture_output=True)
    
    # Push
    push_result = subprocess.run(["git", "push", "origin", branch], capture_output=True, text=True)
    
    if push_result.returncode == 0:
        return True, "Push successful"
    else:
        return False, f"Push failed: {push_result.stderr}"


# Main execution
print("="*60)
print("🧪 TESTING GIT PUSH CAPABILITY")
print("="*60)

try:
    # Change to repo directory
    os.chdir(REPO_DIR)
    
    # Create test log
    log_path, timestamp = create_test_log()
    print(f"✅ Created detailed test log: {log_path}")
    
    # Commit and push
    commit_msg = f"Test: Kaggle GPU verification - {timestamp.strftime('%Y-%m-%d %H:%M:%S')}"
    success, message = git_commit_and_push(log_path, commit_msg, BRANCH)
    
    if success:
        print("\n✅ Successfully pushed to GitHub!")
        print(f"   📁 Check: {log_path}")
        print(f"   📅 Pushed at: {timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
        print("   💡 Pull on your local machine to verify sync")
    else:
        print(f"\n❌ {message}")
        
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    # Always return to parent directory
    os.chdir("..")

print("="*60)

🧪 TESTING GIT PUSH CAPABILITY
✅ Created detailed test log: kaggle_logs/connection_test.log

✅ Successfully pushed to GitHub!
   📁 Check: kaggle_logs/connection_test.log
   📅 Pushed at: 2025-12-18 15:14:37
   💡 Pull on your local machine to verify sync


# 🔗 Step 5: Integrate vLLM with Backend

**This connects the loaded vLLM model to your backend code**

In [14]:
# Register the vLLM model with the backend
import sys
sys.path.insert(0, os.path.join(REPO_DIR, "backend"))

from src.services.vllm_client import VLLMClient

print("="*60)
print("🔗 INTEGRATING vLLM WITH BACKEND")
print("="*60)

# Register the globally loaded vLLM model
VLLMClient.set_model(llm)

print("✅ vLLM model is now available to backend services")
print(f"   Model: {model_path}")
print(f"   GPUs: {torch.cuda.device_count()}")
print(f"   Backend will use LLM_BACKEND={os.getenv('LLM_BACKEND')}")
print("="*60)

🔗 INTEGRATING vLLM WITH BACKEND
✅ vLLM model registered with VLLMClient
✅ vLLM model is now available to backend services
   Model: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
   GPUs: 2
   Backend will use LLM_BACKEND=vllm


In [15]:
pip install -r /kaggle/working/Subchat-Trees/backend/requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/67.3 kB ? eta -:--:--Collecting pypika>=0.48.9 (from chromadb>=0.4.0->-r /kaggle/working/Subchat-Trees/backend/requirements.txt (line 46))
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/4.2 MB 46.3 MB/s eta 0:00:01Requirement already satisfied: pyasn1<0.7.0,>=0.6.1 in /usr/local/lib/python3.11/dist-packages (from pyasn1-modules>=0.2.1->google-auth>=1.0.1->kubernetes>=28.1.0->chromadb>=0.4.0->-r /kaggle/working/Subchat-Trees/backend/requirements.txt (line 46)) (0.6.1)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2

# 🧪 Step 6: Test vLLM Integration

**Quick test to verify backend can use vLLM on Kaggle GPU**

In [16]:
# Test the backend with vLLM
from src.services.simple_llm import SimpleLLMClient
from src.models.tree import TreeNode

print("="*60)
print("🧪 TESTING BACKEND WITH vLLM")
print("="*60)

# Create a simple test
llm_client = SimpleLLMClient()

# Create a test node (TreeNode creates its own buffer internally)
# Note: TreeNode uses 'node_id' (not 'id'), and buffer_size (not buffer object)
root = TreeNode(
    node_id="test", 
    title="Test Conversation", 
    buffer_size=5,
    llm_client=llm_client
)

# Add a message to the node's buffer
root.buffer.add_message("user", "Hello, test message")

# Test generation
print("\n📝 Testing response generation...")
response = llm_client.generate_response(root, "What is 2+2?")

print(f"\n✅ Response: {response}")
print(f"📊 Token usage: {llm_client.get_last_usage()}")
print("\n" + "="*60)
print("✅ Backend integration successful!")
print("="*60)

✅ Using vLLM backend with Kaggle GPU: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
🧪 TESTING BACKEND WITH vLLM
✅ vLLM connected. Using Kaggle GPU
📊 Buffer size: 5 messages | Summarization will trigger every 5 messages
📋 Buffer (1/5): Last 3 messages (full log in file)
   1. [user] Hello, test message

📝 Testing response generation...
*******************context*********************
 [{'role': 'user', 'content': 'Hello, test message'}]


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


✅ Response: Hello! It looks like you might be testing or starting a conversation. How can I assist you today? If you have any specific questions or need help with something, feel free to ask!
📊 Token usage: {'prompt_tokens': 23, 'completion_tokens': 42, 'total_tokens': 112}

✅ Backend integration successful!


# testing ends

# 🚀 Step 7: Start Backend Server

**Run the FastAPI server with vLLM backend on Kaggle GPU**

In [17]:
print("🚀 SETUP COMPLETE - READY FOR SUBCHAT TREES EXECUTION")

🚀 SETUP COMPLETE - READY FOR SUBCHAT TREES EXECUTION


In [18]:
pwd

'/kaggle/working'

In [ ]:
# ⚠️ IMPORTANT: vLLM model must be registered in the SAME process as the server
# We'll use nest_asyncio to allow uvicorn to run in Jupyter's existing event loop

import uvicorn
import os
import nest_asyncio

# Allow nested event loops (required for Jupyter)
nest_asyncio.apply()

# Ensure backend is in path
import sys
sys.path.insert(0, f"/kaggle/working/{REPO_DIR}/backend")

# Re-register vLLM model (in case it was lost)
from src.services.vllm_client import VLLMClient
VLLMClient.set_model(llm)

print("="*60)
print("🚀 STARTING BACKEND SERVER IN NOTEBOOK KERNEL")
print("="*60)
print(f"📂 Backend path: /kaggle/working/{REPO_DIR}/backend")
print(f"🔧 Backend mode: {os.getenv('LLM_BACKEND')}")
print(f"✅ vLLM model registered: {VLLMClient.is_available()}")
print(f"🎯 Server URL: http://0.0.0.0:8000")
print("="*60)
print("\n⚠️  Server starting... (will block this cell)")
print("💡 Stop with: Kernel → Interrupt\n")

# Change to backend directory
os.chdir(f"/kaggle/working/{REPO_DIR}/backend")

# Start the server programmatically (same process, can access llm variable)
# nest_asyncio allows this to work in Jupyter's existing event loop
uvicorn.run("src.main:app", host="0.0.0.0", port=8000, reload=False)

✅ vLLM model registered with VLLMClient
🚀 STARTING BACKEND SERVER IN NOTEBOOK KERNEL
📂 Backend path: /kaggle/working/Subchat-Trees/backend
🔧 Backend mode: vllm
✅ vLLM model registered: True
🎯 Server URL: http://0.0.0.0:8000

⚠️  Server starting... (will block this cell)
💡 Stop with: Kernel → Interrupt

✅ vLLM connected. Using Kaggle GPU


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Traceback (most recent call last):
  File "/kaggle/working/Subchat-Trees/backend/src/models/tree.py", line 191, in _create_rolling_summary
    response = self.llm_client.groq_client.chat.completions.create(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/groq/resources/chat/completions.py", line 461, in create
    return self._post(
           ^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/groq/_base_client.py", line 1242, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/groq/_base_client.py", line 1044, in request
    raise self._make_status_error_from_response(err.response) from None
groq.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

⚠️  Failed to archive message: Error getting collection: Collection [6bbb2613-c537-4b7e-9d02-35f79d45c390] does not exist.
💾 Indexed: [assistant] No, I'm not capable of experiencing emot...
🔄 Buffer full - evicting: I appreciate your dedication. Thank you ...
📋 Buffer (5/5): Last 3 messages (full log in file)
   1. [assistant] Currently, AI systems like myself do not have the ...
   2. [user] Your answer became super slow on my screen. Are yo...
   3. [assistant] No, I'm not capable of experiencing emotions like ...
INFO:     127.0.0.1:52518 - "POST /api/conversations/3a383ea4-2aef-42bb-b051-d1b9c2847bea/messages HTTP/1.1" 200 OK
⚠️  Failed to archive message: Error getting collection: Collection [6bbb2613-c537-4b7e-9d02-35f79d45c390] does not exist.
💾 Indexed: [user] Would you like to have the ability to ex...
🔄 Buffer full - evicting: Certainly! As an artificial intelligence...
📋 Buffer (5/5): Last 3 messages (full log in file)
   1. [user] Your answer became super slow on my screen.

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

(VllmWorkerProcess pid=158) INFO 12-18 16:02:10 [multiproc_worker_utils.py:259] Worker exiting
INFO 12-18 16:02:13 [multiproc_worker_utils.py:124] Killing local vLLM worker processes
